In [2]:
## Import modules
import pandas as pd
import numpy as np
import scipy.sparse as sps
import matplotlib.pyplot as plt
import os
#import progressbar as pp
import re
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import random
import pickle
import gensim

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

/home/ruihaoqiu/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [7]:
logger.info("## load data - 1 million in total")
#job_description_df = pd.read_csv("/mnt/c/Users/ruihao/Google Drive/Data/job_description_all_1m.csv")
#job_description_df = pd.read_csv("/Users/qiuruihao/Google Drive (ruihqiu@gmail.com)/Data/job_description_all_1m.csv")

2018-10-16 10:10:21,947 : INFO : ## load data - 1 million in total


In [3]:
## pre-analysis
logger.info("# Extract english job descriptions - 400k in total")

# filter out short job description (<100 words)
job_description_en_df = pd.DataFrame(job_description
                                     _df[job_description_df.language_id==1]["job_description"].dropna())
job_description_en_df = job_description_en_df[job_description_en_df["job_description"].apply(lambda x: len(x.split())>100)]

In [8]:
def load_stopwords(stopfile):
    return set(line.strip() for line in open(stopfile))

logger.info("Load stopwords")
stopwords = load_stopwords("stopwords_en.txt")

2018-10-16 10:12:08,510 : INFO : Load stopwords


In [86]:
## An example
n = random.randint(0, len(job_description_en_df))
example = job_description_en_df.iloc[n].values[0]
print(example)

* Salary: Competitive
  * Location: Singapore 
  * Job Type: Permanent, Full time
  * Company: Citibank NA
  * Updated on: 2018-08-24

## Head of Core Warehousing and Big Data Architecture Global Consumer
Technology

Head of Core Warehousing and Big Data Architecture Global Consumer Technology

  * **Primary Location:** Singapore,Singapore,Singapore
  * **Education:** Bachelor's Degree
  * **Job Function:** Technology
  * **Schedule:** Full-time
  * **Shift:** Day Job
  * **Employee Status:** Regular
  * **Travel Time:** Yes, 10 % of the Time
  * **Job ID:** 18047959

  
  
**Description**  
  
**Position Summary:**  
  

  * The Chief Technology Officer (CTO) function is responsible for strategic and operational management of the technology environment supporting Citi's Global Consumer Technology (GCT) function. Key responsibilities of the CTO include: enterprise architecture governance; technology innovation, strategy and roadmap development; software development technology, process 

In [87]:
def paragraph_segment(text):
    text = re.sub(r"[\(\>]http.+?(\s|\n.+?)*[\)\>]", " ", text, re.MULTILINE) ## remove url
    text = re.split("\n[\s]*\n", text) ## segment with double newlines
    text = [re.sub(r"[^\w\&,.;!:()-]", " ", x) for x in text] ## remove special characters
    text = [re.sub(r"\s+", " ", x.strip()) for x in text] ## remove spaces
    text = list(filter(lambda x: len(x.split())>3, text)) ## remove short sentences
    return text

paragraph_segment(example)

['Salary: Competitive Location: Singapore Job Type: Permanent, Full time Company: Citibank NA Updated on: 2018-08-24',
 'Head of Core Warehousing and Big Data Architecture Global Consumer Technology',
 'Head of Core Warehousing and Big Data Architecture Global Consumer Technology',
 'Primary Location: Singapore,Singapore,Singapore Education: Bachelor s Degree Job Function: Technology Schedule: Full-time Shift: Day Job Employee Status: Regular Travel Time: Yes, 10 of the Time Job ID: 18047959',
 'The Chief Technology Officer (CTO) function is responsible for strategic and operational management of the technology environment supporting Citi s Global Consumer Technology (GCT) function. Key responsibilities of the CTO include: enterprise architecture governance; technology innovation, strategy and roadmap development; software development technology, process and design standards; performance engineering and testing; development operations (devops) strategy and execution; operational manage

In [91]:
test_text = "For our client, a dynamically growing organization from the financial sector, we are looking for a person for the position"
stopword_regex = re.compile(r"\b(" + "|".join(stopwords) + r")\b", re.IGNORECASE)

def char_splitter(text):
    char_ = re.compile("[.,;!:()_-]")
    sentences = re.split(char_, text)
    return [x.strip() for x in sentences if x.strip()]

def stopword_splitter(stopword_regex, text):
    sentence = re.sub(stopword_regex, ";", text)
    phrases = re.split(r";+", sentence)
    phrases_output = [x.strip() for x in phrases if x.strip()]
    return phrases_output

def splitter(paragraph_list):
    output_list = []
    long_sentences = []
    for p in paragraph_list:
        long_sentences += char_splitter(p)
    for sent in long_sentences:
        phrases = stopword_splitter(stopword_regex, sent)
        for phrase in phrases:
            if len(phrase.split()) > 3:
                output_list += phrase.split()
            else:
                output_list.append(phrase)
                
    return output_list
  
def remove_numbers(text_list):
    return [x for x in text_list if not x.isdigit() and len(x)>1 or x in ["C", "R"]]

def get_lower_case(text_list):
    case_sensitive_words = ["R", "C", "IT", "MS"]
    return [x.lower() for x in text_list if not x in case_sensitive_words]

word_list = splitter(paragraph_segment(example))
print(get_lower_case(remove_numbers(word_list)))

['salary', 'competitive location', 'singapore job type', 'permanent', 'time company', 'citibank na updated', 'head', 'core warehousing', 'big', 'data', 'architecture', 'global', 'consumer', 'technology', 'head', 'core warehousing', 'big', 'data', 'architecture', 'global', 'consumer', 'technology', 'primary location', 'singapore', 'singapore', 'singapore education', 'bachelor', 'degree', 'job', 'function', 'technology schedule', 'time shift', 'day', 'job', 'employee', 'status', 'regular travel time', 'yes', 'time job id', 'chief technology officer', 'cto', 'function', 'responsible', 'strategic', 'operational management', 'technology', 'environment', 'supporting', 'citi', 'global', 'consumer', 'technology', 'gct', 'function', 'key responsibilities', 'cto include', 'enterprise architecture governance', 'technology innovation', 'strategy', 'roadmap development', 'software development technology', 'process', 'design standards', 'performance engineering', 'testing', 'development operations',

In [92]:
logger.info("## save tokenized text")

tokenized_text = []
for n in tqdm(range(len(job_description_en_df))):
    job = job_description_en_df.iloc[n].values[0]
    tokenized_text.append((get_lower_case(remove_numbers(splitter(paragraph_segment(job))))))

2018-10-16 12:31:46,724 : INFO : ## save tokenized text
100%|██████████| 395434/395434 [43:36<00:00, 151.15it/s]


In [94]:
import pickle

with open('tokenized_text.pkl', 'wb') as f:
    pickle.dump(tokenized_text, f)

In [95]:
model = gensim.models.Word2Vec(tokenized_text, size=500, window=10, min_count=10)
model.save("word2vec_ngram.model")

2018-10-16 14:19:34,601 : INFO : collecting all words and their counts
2018-10-16 14:19:34,616 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-10-16 14:19:36,488 : INFO : PROGRESS: at sentence #10000, processed 1919148 words, keeping 172388 word types
2018-10-16 14:19:38,339 : INFO : PROGRESS: at sentence #20000, processed 4876321 words, keeping 297883 word types
2018-10-16 14:19:40,125 : INFO : PROGRESS: at sentence #30000, processed 7064595 words, keeping 475281 word types
2018-10-16 14:19:41,592 : INFO : PROGRESS: at sentence #40000, processed 9009862 words, keeping 565150 word types
2018-10-16 14:19:43,442 : INFO : PROGRESS: at sentence #50000, processed 11115154 words, keeping 619148 word types
2018-10-16 14:19:44,877 : INFO : PROGRESS: at sentence #60000, processed 13285199 words, keeping 679274 word types
2018-10-16 14:19:47,063 : INFO : PROGRESS: at sentence #70000, processed 15746285 words, keeping 737473 word types
2018-10-16 14:19:48,735 : INF

2018-10-16 14:27:04,147 : INFO : EPOCH 1 - PROGRESS: at 5.65% examples, 133882 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:27:05,234 : INFO : EPOCH 1 - PROGRESS: at 5.89% examples, 135393 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:27:06,339 : INFO : EPOCH 1 - PROGRESS: at 6.10% examples, 136111 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:27:07,410 : INFO : EPOCH 1 - PROGRESS: at 6.37% examples, 138203 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:27:08,462 : INFO : EPOCH 1 - PROGRESS: at 6.79% examples, 142642 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:27:09,476 : INFO : EPOCH 1 - PROGRESS: at 7.15% examples, 147151 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:27:10,490 : INFO : EPOCH 1 - PROGRESS: at 7.56% examples, 151694 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:27:11,502 : INFO : EPOCH 1 - PROGRESS: at 8.00% examples, 156096 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:27:12,567 : INFO : EPOCH 1 - PROGRESS: at 8.36% examples, 158717 words/s, in_qsize 5, out_

2018-10-16 14:28:23,182 : INFO : EPOCH 1 - PROGRESS: at 33.79% examples, 232803 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:28:24,199 : INFO : EPOCH 1 - PROGRESS: at 34.18% examples, 233167 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:28:25,226 : INFO : EPOCH 1 - PROGRESS: at 34.61% examples, 233894 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:28:26,283 : INFO : EPOCH 1 - PROGRESS: at 34.97% examples, 234359 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:28:27,302 : INFO : EPOCH 1 - PROGRESS: at 35.31% examples, 234736 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:28:28,302 : INFO : EPOCH 1 - PROGRESS: at 35.67% examples, 235405 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:28:29,408 : INFO : EPOCH 1 - PROGRESS: at 36.07% examples, 235621 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:28:30,426 : INFO : EPOCH 1 - PROGRESS: at 36.38% examples, 235382 words/s, in_qsize 3, out_qsize 0
2018-10-16 14:28:31,448 : INFO : EPOCH 1 - PROGRESS: at 36.77% examples, 235636 words/s, in_qsiz

2018-10-16 14:29:40,806 : INFO : EPOCH 1 - PROGRESS: at 61.14% examples, 248659 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:29:41,838 : INFO : EPOCH 1 - PROGRESS: at 61.57% examples, 249010 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:29:42,873 : INFO : EPOCH 1 - PROGRESS: at 62.03% examples, 249378 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:29:43,878 : INFO : EPOCH 1 - PROGRESS: at 62.43% examples, 249690 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:29:44,901 : INFO : EPOCH 1 - PROGRESS: at 62.78% examples, 249997 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:29:45,924 : INFO : EPOCH 1 - PROGRESS: at 63.22% examples, 250323 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:29:46,939 : INFO : EPOCH 1 - PROGRESS: at 63.63% examples, 250612 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:29:47,947 : INFO : EPOCH 1 - PROGRESS: at 63.97% examples, 250672 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:29:48,949 : INFO : EPOCH 1 - PROGRESS: at 64.24% examples, 250663 words/s, in_qsiz

2018-10-16 14:30:55,852 : INFO : EPOCH 1 - PROGRESS: at 87.62% examples, 259879 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:30:56,858 : INFO : EPOCH 1 - PROGRESS: at 87.99% examples, 259970 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:30:57,870 : INFO : EPOCH 1 - PROGRESS: at 88.20% examples, 259621 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:30:58,919 : INFO : EPOCH 1 - PROGRESS: at 88.64% examples, 259845 words/s, in_qsize 6, out_qsize 1
2018-10-16 14:30:59,931 : INFO : EPOCH 1 - PROGRESS: at 89.07% examples, 259996 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:31:00,948 : INFO : EPOCH 1 - PROGRESS: at 89.48% examples, 260113 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:31:01,969 : INFO : EPOCH 1 - PROGRESS: at 89.86% examples, 260260 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:31:02,974 : INFO : EPOCH 1 - PROGRESS: at 90.26% examples, 260394 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:31:04,004 : INFO : EPOCH 1 - PROGRESS: at 90.67% examples, 260559 words/s, in_qsiz

...

2018-10-16 14:43:39,174 : INFO : EPOCH 4 - PROGRESS: at 67.69% examples, 297285 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:43:40,338 : INFO : EPOCH 4 - PROGRESS: at 67.97% examples, 296692 words/s, in_qsize 5, out_qsize 1
2018-10-16 14:43:41,477 : INFO : EPOCH 4 - PROGRESS: at 68.07% examples, 295226 words/s, in_qsize 4, out_qsize 0
2018-10-16 14:43:42,508 : INFO : EPOCH 4 - PROGRESS: at 68.44% examples, 295263 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:43:43,527 : INFO : EPOCH 4 - PROGRESS: at 68.83% examples, 295318 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:43:44,575 : INFO : EPOCH 4 - PROGRESS: at 69.23% examples, 295447 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:43:45,602 : INFO : EPOCH 4 - PROGRESS: at 69.60% examples, 295503 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:43:46,628 : INFO : EPOCH 4 - PROGRESS: at 69.97% examples, 295686 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:43:47,654 : INFO : EPOCH 4 - PROGRESS: at 70.31% examples, 295600 words/s, in_qsiz

2018-10-16 14:44:53,958 : INFO : EPOCH 4 - PROGRESS: at 96.48% examples, 299757 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:44:54,978 : INFO : EPOCH 4 - PROGRESS: at 96.94% examples, 299789 words/s, in_qsize 4, out_qsize 1
2018-10-16 14:44:55,991 : INFO : EPOCH 4 - PROGRESS: at 97.38% examples, 299851 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:44:57,006 : INFO : EPOCH 4 - PROGRESS: at 97.85% examples, 299957 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:44:58,055 : INFO : EPOCH 4 - PROGRESS: at 98.21% examples, 299872 words/s, in_qsize 5, out_qsize 1
2018-10-16 14:44:59,061 : INFO : EPOCH 4 - PROGRESS: at 98.64% examples, 299975 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:45:00,069 : INFO : EPOCH 4 - PROGRESS: at 99.02% examples, 299928 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:45:01,099 : INFO : EPOCH 4 - PROGRESS: at 99.26% examples, 299336 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:45:02,131 : INFO : EPOCH 4 - PROGRESS: at 99.73% examples, 299450 words/s, in_qsiz

2018-10-16 14:46:11,330 : INFO : EPOCH 5 - PROGRESS: at 23.73% examples, 257561 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:46:12,350 : INFO : EPOCH 5 - PROGRESS: at 24.19% examples, 258800 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:46:13,353 : INFO : EPOCH 5 - PROGRESS: at 24.63% examples, 259889 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:46:14,355 : INFO : EPOCH 5 - PROGRESS: at 25.02% examples, 261316 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:46:15,383 : INFO : EPOCH 5 - PROGRESS: at 25.45% examples, 262381 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:46:16,390 : INFO : EPOCH 5 - PROGRESS: at 25.83% examples, 262991 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:46:17,390 : INFO : EPOCH 5 - PROGRESS: at 26.15% examples, 263826 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:46:18,399 : INFO : EPOCH 5 - PROGRESS: at 26.48% examples, 264300 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:46:19,415 : INFO : EPOCH 5 - PROGRESS: at 26.88% examples, 264995 words/s, in_qsiz

2018-10-16 14:47:25,642 : INFO : EPOCH 5 - PROGRESS: at 53.04% examples, 284623 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:47:26,657 : INFO : EPOCH 5 - PROGRESS: at 53.39% examples, 284824 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:47:27,687 : INFO : EPOCH 5 - PROGRESS: at 53.80% examples, 285086 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:47:28,696 : INFO : EPOCH 5 - PROGRESS: at 54.22% examples, 285400 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:47:29,715 : INFO : EPOCH 5 - PROGRESS: at 54.59% examples, 285314 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:47:30,717 : INFO : EPOCH 5 - PROGRESS: at 55.04% examples, 285370 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:47:31,728 : INFO : EPOCH 5 - PROGRESS: at 55.29% examples, 284847 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:47:32,737 : INFO : EPOCH 5 - PROGRESS: at 55.71% examples, 285328 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:47:33,743 : INFO : EPOCH 5 - PROGRESS: at 56.14% examples, 285730 words/s, in_qsiz

2018-10-16 14:48:43,850 : INFO : EPOCH 5 - PROGRESS: at 81.06% examples, 283036 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:48:44,866 : INFO : EPOCH 5 - PROGRESS: at 81.49% examples, 283299 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:48:45,868 : INFO : EPOCH 5 - PROGRESS: at 81.91% examples, 283589 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:48:46,904 : INFO : EPOCH 5 - PROGRESS: at 82.27% examples, 283923 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:48:47,919 : INFO : EPOCH 5 - PROGRESS: at 82.72% examples, 284184 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:48:48,923 : INFO : EPOCH 5 - PROGRESS: at 83.13% examples, 284511 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:48:49,926 : INFO : EPOCH 5 - PROGRESS: at 83.45% examples, 284968 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:48:50,954 : INFO : EPOCH 5 - PROGRESS: at 83.81% examples, 285297 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:48:51,960 : INFO : EPOCH 5 - PROGRESS: at 84.30% examples, 285574 words/s, in_qsiz

In [96]:
model = gensim.models.Word2Vec.load("word2vec_ngram.model")
model.train(tokenized_text, total_examples=len(tokenized_text), epochs=10)

2018-10-16 14:49:49,729 : INFO : loading Word2Vec object from word2vec_ngram.model
2018-10-16 14:53:31,621 : INFO : loading wv recursively from word2vec_ngram.model.wv.* with mmap=None
2018-10-16 14:53:31,656 : INFO : loading vectors from word2vec_ngram.model.wv.vectors.npy with mmap=None
2018-10-16 14:53:33,850 : INFO : setting ignored attribute vectors_norm to None
2018-10-16 14:53:33,855 : INFO : loading vocabulary recursively from word2vec_ngram.model.vocabulary.* with mmap=None
2018-10-16 14:53:33,858 : INFO : loading trainables recursively from word2vec_ngram.model.trainables.* with mmap=None
2018-10-16 14:53:33,861 : INFO : loading syn1neg from word2vec_ngram.model.trainables.syn1neg.npy with mmap=None
2018-10-16 14:53:35,716 : INFO : setting ignored attribute cum_table to None
2018-10-16 14:53:35,724 : INFO : loaded word2vec_ngram.model
2018-10-16 14:53:37,778 : WARNING : Effective 'alpha' higher than previous training cycles
2018-10-16 14:53:37,779 : INFO : training model with

2018-10-16 14:54:52,124 : INFO : EPOCH 1 - PROGRESS: at 25.74% examples, 259903 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:54:53,128 : INFO : EPOCH 1 - PROGRESS: at 26.10% examples, 261126 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:54:54,128 : INFO : EPOCH 1 - PROGRESS: at 26.47% examples, 262136 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:54:55,165 : INFO : EPOCH 1 - PROGRESS: at 26.88% examples, 262907 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:54:56,167 : INFO : EPOCH 1 - PROGRESS: at 27.35% examples, 263795 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:54:57,186 : INFO : EPOCH 1 - PROGRESS: at 27.82% examples, 264897 words/s, in_qsize 6, out_qsize 1
2018-10-16 14:54:58,190 : INFO : EPOCH 1 - PROGRESS: at 28.26% examples, 265711 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:54:59,228 : INFO : EPOCH 1 - PROGRESS: at 28.69% examples, 266016 words/s, in_qsize 4, out_qsize 1
2018-10-16 14:55:00,267 : INFO : EPOCH 1 - PROGRESS: at 29.11% examples, 266374 words/s, in_qsiz

2018-10-16 14:56:10,292 : INFO : EPOCH 1 - PROGRESS: at 55.93% examples, 282060 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:56:11,316 : INFO : EPOCH 1 - PROGRESS: at 56.41% examples, 282381 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:56:12,328 : INFO : EPOCH 1 - PROGRESS: at 56.97% examples, 282900 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:56:13,332 : INFO : EPOCH 1 - PROGRESS: at 57.55% examples, 283326 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:56:14,335 : INFO : EPOCH 1 - PROGRESS: at 58.10% examples, 283816 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:56:15,358 : INFO : EPOCH 1 - PROGRESS: at 58.61% examples, 284228 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:56:16,377 : INFO : EPOCH 1 - PROGRESS: at 59.12% examples, 284621 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:56:17,401 : INFO : EPOCH 1 - PROGRESS: at 59.58% examples, 284894 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:56:18,456 : INFO : EPOCH 1 - PROGRESS: at 60.01% examples, 284950 words/s, in_qsiz

2018-10-16 14:57:33,217 : INFO : EPOCH 1 - PROGRESS: at 84.14% examples, 277664 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:57:34,265 : INFO : EPOCH 1 - PROGRESS: at 84.58% examples, 277913 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:57:35,286 : INFO : EPOCH 1 - PROGRESS: at 84.99% examples, 278099 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:57:36,289 : INFO : EPOCH 1 - PROGRESS: at 85.46% examples, 278320 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:57:37,321 : INFO : EPOCH 1 - PROGRESS: at 85.81% examples, 278165 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:57:38,354 : INFO : EPOCH 1 - PROGRESS: at 86.00% examples, 277676 words/s, in_qsize 5, out_qsize 1
2018-10-16 14:57:42,691 : INFO : EPOCH 1 - PROGRESS: at 86.04% examples, 272906 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:57:43,710 : INFO : EPOCH 1 - PROGRESS: at 86.33% examples, 272661 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:57:44,716 : INFO : EPOCH 1 - PROGRESS: at 86.81% examples, 273074 words/s, in_qsiz

2018-10-16 14:58:54,920 : INFO : EPOCH 2 - PROGRESS: at 11.95% examples, 220932 words/s, in_qsize 6, out_qsize 2
2018-10-16 14:58:55,924 : INFO : EPOCH 2 - PROGRESS: at 12.43% examples, 224786 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:58:56,933 : INFO : EPOCH 2 - PROGRESS: at 12.89% examples, 227874 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:58:57,968 : INFO : EPOCH 2 - PROGRESS: at 13.38% examples, 231665 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:58:58,989 : INFO : EPOCH 2 - PROGRESS: at 13.83% examples, 235197 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:59:00,040 : INFO : EPOCH 2 - PROGRESS: at 14.27% examples, 238232 words/s, in_qsize 5, out_qsize 0
2018-10-16 14:59:01,081 : INFO : EPOCH 2 - PROGRESS: at 14.77% examples, 240699 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:59:02,095 : INFO : EPOCH 2 - PROGRESS: at 15.28% examples, 242936 words/s, in_qsize 6, out_qsize 0
2018-10-16 14:59:03,116 : INFO : EPOCH 2 - PROGRESS: at 15.80% examples, 244573 words/s, in_qsiz

...

2018-10-16 15:31:46,674 : INFO : EPOCH 9 - PROGRESS: at 91.81% examples, 313265 words/s, in_qsize 5, out_qsize 0
2018-10-16 15:31:47,699 : INFO : EPOCH 9 - PROGRESS: at 92.26% examples, 313357 words/s, in_qsize 5, out_qsize 0
2018-10-16 15:31:48,709 : INFO : EPOCH 9 - PROGRESS: at 92.72% examples, 313485 words/s, in_qsize 5, out_qsize 0
2018-10-16 15:31:49,716 : INFO : EPOCH 9 - PROGRESS: at 93.21% examples, 313601 words/s, in_qsize 6, out_qsize 0
2018-10-16 15:31:50,748 : INFO : EPOCH 9 - PROGRESS: at 93.74% examples, 313807 words/s, in_qsize 6, out_qsize 0
2018-10-16 15:31:51,848 : INFO : EPOCH 9 - PROGRESS: at 94.13% examples, 313579 words/s, in_qsize 4, out_qsize 1
2018-10-16 15:31:52,876 : INFO : EPOCH 9 - PROGRESS: at 94.48% examples, 313282 words/s, in_qsize 6, out_qsize 1
2018-10-16 15:31:53,890 : INFO : EPOCH 9 - PROGRESS: at 94.99% examples, 313466 words/s, in_qsize 6, out_qsize 0
2018-10-16 15:31:54,899 : INFO : EPOCH 9 - PROGRESS: at 95.47% examples, 313666 words/s, in_qsiz

2018-10-16 15:32:57,268 : INFO : EPOCH 10 - PROGRESS: at 22.36% examples, 319671 words/s, in_qsize 5, out_qsize 0
2018-10-16 15:32:58,284 : INFO : EPOCH 10 - PROGRESS: at 22.78% examples, 319919 words/s, in_qsize 5, out_qsize 0
2018-10-16 15:32:59,296 : INFO : EPOCH 10 - PROGRESS: at 23.23% examples, 320385 words/s, in_qsize 5, out_qsize 0
2018-10-16 15:33:00,320 : INFO : EPOCH 10 - PROGRESS: at 23.64% examples, 320540 words/s, in_qsize 5, out_qsize 0
2018-10-16 15:33:01,323 : INFO : EPOCH 10 - PROGRESS: at 24.09% examples, 321069 words/s, in_qsize 6, out_qsize 0
2018-10-16 15:33:02,366 : INFO : EPOCH 10 - PROGRESS: at 24.56% examples, 321118 words/s, in_qsize 5, out_qsize 0
2018-10-16 15:33:03,374 : INFO : EPOCH 10 - PROGRESS: at 24.93% examples, 321624 words/s, in_qsize 6, out_qsize 0
2018-10-16 15:33:04,394 : INFO : EPOCH 10 - PROGRESS: at 25.34% examples, 321756 words/s, in_qsize 5, out_qsize 0
2018-10-16 15:33:05,395 : INFO : EPOCH 10 - PROGRESS: at 25.73% examples, 321548 words/s

2018-10-16 15:34:15,587 : INFO : EPOCH 10 - PROGRESS: at 51.65% examples, 303749 words/s, in_qsize 5, out_qsize 0
2018-10-16 15:34:16,592 : INFO : EPOCH 10 - PROGRESS: at 52.03% examples, 303462 words/s, in_qsize 5, out_qsize 0
2018-10-16 15:34:17,604 : INFO : EPOCH 10 - PROGRESS: at 52.43% examples, 303977 words/s, in_qsize 5, out_qsize 0
2018-10-16 15:34:18,649 : INFO : EPOCH 10 - PROGRESS: at 52.76% examples, 304107 words/s, in_qsize 6, out_qsize 0
2018-10-16 15:34:19,656 : INFO : EPOCH 10 - PROGRESS: at 53.24% examples, 304337 words/s, in_qsize 5, out_qsize 0
2018-10-16 15:34:20,667 : INFO : EPOCH 10 - PROGRESS: at 53.66% examples, 304691 words/s, in_qsize 6, out_qsize 0
2018-10-16 15:34:21,687 : INFO : EPOCH 10 - PROGRESS: at 54.12% examples, 305110 words/s, in_qsize 5, out_qsize 0
2018-10-16 15:34:22,717 : INFO : EPOCH 10 - PROGRESS: at 54.59% examples, 305422 words/s, in_qsize 6, out_qsize 0
2018-10-16 15:34:23,749 : INFO : EPOCH 10 - PROGRESS: at 55.08% examples, 305520 words/s

2018-10-16 15:35:29,274 : INFO : EPOCH 10 - PROGRESS: at 82.39% examples, 312702 words/s, in_qsize 5, out_qsize 0
2018-10-16 15:35:30,293 : INFO : EPOCH 10 - PROGRESS: at 82.81% examples, 312797 words/s, in_qsize 5, out_qsize 0
2018-10-16 15:35:31,299 : INFO : EPOCH 10 - PROGRESS: at 83.17% examples, 312835 words/s, in_qsize 5, out_qsize 0
2018-10-16 15:35:32,302 : INFO : EPOCH 10 - PROGRESS: at 83.49% examples, 313153 words/s, in_qsize 5, out_qsize 0
2018-10-16 15:35:33,342 : INFO : EPOCH 10 - PROGRESS: at 83.77% examples, 313000 words/s, in_qsize 5, out_qsize 0
2018-10-16 15:35:34,367 : INFO : EPOCH 10 - PROGRESS: at 84.21% examples, 312937 words/s, in_qsize 6, out_qsize 0
2018-10-16 15:35:35,382 : INFO : EPOCH 10 - PROGRESS: at 84.66% examples, 313139 words/s, in_qsize 5, out_qsize 0
2018-10-16 15:35:36,420 : INFO : EPOCH 10 - PROGRESS: at 85.11% examples, 313253 words/s, in_qsize 5, out_qsize 0
2018-10-16 15:35:37,439 : INFO : EPOCH 10 - PROGRESS: at 85.57% examples, 313350 words/s

(772711823, 890730540)

In [124]:
## testing

word_vectors = model.wv

test_list_1gram = ["python", "java", "engineer", "excel", "git", "automobile", "finance", "english", "german", "vehicle"]
test_list_2gram = ["machine learning", "project management", "risk management", "data science", "statistical modelling", "big data", "automobile engineering", "human resource"]
test_list_3gram = ["natural language processing", "quality assurance engineering", "ms office excel"]

test_list = test_list_1gram + test_list_2gram + test_list_3gram

for word in test_list:
    print(word, ":\n", word_vectors.most_similar(word, topn=10))

python :
 [('java', 0.5661136507987976), ('programming', 0.5485414266586304), ('sql', 0.5414139032363892), ('scripting languages', 0.5005762577056885), ('scripting', 0.4916435480117798), ('c c', 0.48072144389152527), ('linux', 0.4796450138092041), ('perl', 0.4746547341346741), ('modern programming language', 0.4569898247718811), ('scala', 0.44549480080604553)]
java :
 [('java j2ee', 0.567360520362854), ('python', 0.5661136507987976), ('sql', 0.5426121950149536), ('programming', 0.5244420766830444), ('linux', 0.5222653150558472), ('html5', 0.5066940188407898), ('spring', 0.49310192465782166), ('angular', 0.49159738421440125), ('web', 0.48923367261886597), ('php', 0.4854477047920227)]
engineer :
 [('developer', 0.5396875739097595), ('engineering', 0.4639178514480591), ('architect', 0.4519370198249817), ('engineers', 0.43471527099609375), ('software engineer', 0.3961600065231323), ('senior engineer', 0.37202188372612), ('specialist', 0.3717525601387024), ('r&d software engineering', 0.365

In [133]:
logger.info("## save word vectors")

from gensim.models import KeyedVectors
from gensim.test.utils import get_tmpfile

fname = get_tmpfile("vectors.kv")
word_vectors.save(fname)
# word_vectors = KeyedVectors.load(fname, mmap='r')

2018-10-16 17:00:25,366 : INFO : ## save word vectors
2018-10-16 17:00:25,371 : INFO : saving Word2VecKeyedVectors object under /tmp/vectors.kv, separately None
2018-10-16 17:00:25,376 : INFO : storing np array 'vectors' to /tmp/vectors.kv.vectors.npy
2018-10-16 17:00:30,336 : INFO : not storing attribute vectors_norm
2018-10-16 17:00:32,662 : INFO : saved /tmp/vectors.kv


In [6]:
from gensim.models import KeyedVectors
from gensim.test.utils import get_tmpfile
fname = get_tmpfile("vectors.kv")
old_word_vectors = KeyedVectors.load(fname, mmap='r')

#for word in test_list:
print("man", ":\n", old_word_vectors.most_similar("man", topn=10))

2018-10-18 15:59:25,834 : INFO : loading Word2VecKeyedVectors object from /tmp/vectors.kv
2018-10-18 15:59:26,854 : INFO : loading vectors from /tmp/vectors.kv.vectors.npy with mmap=r
2018-10-18 15:59:26,867 : INFO : setting ignored attribute vectors_norm to None
2018-10-18 15:59:26,868 : INFO : loaded /tmp/vectors.kv
2018-10-18 15:59:27,011 : INFO : precomputing L2-norms of word weight vectors


man :
 [('flysquare solution basically', 0.31119444966316223), ('jobs matching energy', 0.30456823110580444), ('managing leaves', 0.2859411835670471), ('gas quantitative strategist', 0.28062254190444946), ('fmcg recruitment', 0.2798054814338684), ('shopping malls', 0.2780197858810425), ('leading names', 0.2614626884460449), ('fabrication technology', 0.2609795928001404), ('jobs matching power', 0.25975075364112854), ('statutory requirements managing', 0.25757238268852234)]


In [11]:
def generate_candidate_phrases(text, stopwords):
    coarse_candidates = char_splitter.split(text.lower())
    candidate_phrases = []
    
    for coarse_phrase in coarse_candidates[:1]:
        words = re.split("\s+", coarse_phrase)
        previous_stop = False
        for w in words:
            if w in stopwords and not previous_stop:
                candidate_phrases.append(";")
                previous_stop = True
            elif w not in stopwords:
                candidate_phrases.append(w)
                previous_stop = False
        candidate_phrases.append(";")

    phrases = re.split(";+", ' '.join(candidate_phrases))
    phrases_output = [x.strip() for x in phrases if x.strip()]
    return phrases_output

In [12]:
for text in paragraph_segment(example):
    print(generate_candidate_phrases(text, stopwords))

['client']
['seo technical team']
['miejsce pracy']
['working', 'client']
['working', 'tech seo managers', 'specialists', 'krakow']
['experiance', 'seo', 'technical audits', 'app store optimisation', 'essential']
['opportunity', 'grow', 'international organization']
['osoby zainteresowane prosimy o przesyłanie aplikacji klikając w przycisk aplikowania']
[]
['zastrzegamy sobie możliwość odpowiedzi tylko na wybrane aplikacje']
['agencja wpisana', 'krajowego rejestru agencji zatrudnienia pod numerem 10504']
['interested', 'add', 'cv', 'agreement']
['consent cpl jobs sp']
